In [ ]:
import os
import preprocessor as prep
import postprocessor as postp
import pandas as pd
import unifloc.uniflocpy.uTools.plotly_workflow as pw
import datetime
from sklearn import metrics

Общие настройки и флаги для работы

In [ ]:
well_name = '507'
chess_file_name = 'Скв. 507 (01.07.2018-31.03.2019).xls'

read_initial_data = False
plot_initial_data = False
create_input_data = True
multiprocessing_on = True
created_input_data_type = 1 # 0 - adaptation input, 1 - restore input with calibation coefficients

Определение путей к данным

In [ ]:
current_path = os.getcwd()
time_mark = datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')
path_to_data = current_path + "\\data\\"
path_to_work_dir = current_path + "\\data\\" + well_name +  "\\"
save_dir_name = 'init_edit'
path_to_save = path_to_work_dir + save_dir_name + '\\'
dirnames_list = []
for (dirpath, dirnames, filenames) in os.walk(path_to_data):
    dirnames_list.extend(dirnames)
    break
print(dirnames_list)

# Обработка исходных данных

In [ ]:

if read_initial_data:
    try:
        os.mkdir(path_to_work_dir + save_dir_name)
    except:
        pass
    data_file_path = path_to_work_dir + well_name + ".csv"
    well_data = pd.read_csv(data_file_path,sep=';', header=None)
    well_data = prep.initial_editing(well_data, well_name)
    well_data = prep.create_edited_df(well_data)
    if plot_initial_data:
        plot_file_path = path_to_save + well_name + '_init_sp.html'
        well_data_traces = pw.create_traces_list_for_all_columms(well_data, 'lines+markers', low_memory = True)
        pw.plot_subplots(well_data_traces, plot_file_path, True)
        
    well_data.to_csv(path_to_save + well_name + "_first_edit.csv")

# Генерация входных данных для адаптации

Задание границ

In [ ]:
left_boundary = [datetime.datetime(2019,2,1)]#, datetime.datetime(2018,11,29)]
right_boundary = [datetime.datetime(2019,2,28)]#, datetime.datetime(2019,2,28)]

Чтение исходных данных со СУ (в шахмоткоподобном виде)

In [ ]:
edited_data_cs = pd.read_csv(path_to_save + well_name + "_first_edit.csv", parse_dates = True, index_col = 'Время')
edited_data_cs = edited_data_cs.resample('3h').mean()
edited_data_cs['Выходная частота ПЧ'] = edited_data_cs['Выходная частота ПЧ'].fillna(method='ffill')
edited_data_cs['Температура на приеме насоса (пласт. жидкость)']  = \
        edited_data_cs['Температура на приеме насоса (пласт. жидкость)'].fillna(method='ffill')
if created_input_data_type == 0:
    edited_data_cs = edited_data_cs.dropna(subset =['Объемный дебит жидкости'])
edited_data_cs = edited_data_cs.fillna(method='ffill')
edited_data_cs['ГФ'] = edited_data_cs['Объемный дебит газа'] / edited_data_cs['Объемный дебит нефти']
edited_data_cs = prep.mark_df_columns(edited_data_cs, 'СУ')

Чтение данных с шахматки

In [ ]:
chess_data = pd.read_excel(path_to_work_dir + chess_file_name)
chess_data.index = pd.to_datetime(chess_data['Дата'], dayfirst = True, format = "%d.%m.%Y", infer_datetime_format=True)
del chess_data['Дата']
chess_data.index.name = 'Время'
chess_data = chess_data[chess_data.columns[5:]]
chess_data = chess_data.resample('3h').last()
chess_data = chess_data.fillna(method='ffill')
chess_data = prep.mark_df_columns(chess_data, 'Ш')

Генерация входных данных для адаптации

In [ ]:
if created_input_data_type == 0:
    input_data_dir_name = 'adapt_input_' + time_mark
    path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
    if create_input_data:
        try:
            os.mkdir(path_to_work_dir + input_data_dir_name)
        except:
            pass
    created_input_data = edited_data_cs.join(chess_data, how = 'outer')
    created_input_data = created_input_data.dropna(subset = ['Объемный дебит жидкости (СУ)'])
    created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
    plot_file_path = path_to_input_data + well_name + '_adapt_input.html'
    input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', low_memory = True)
    pw.plot_subplots(input_data_traces, plot_file_path, True)
    
    created_input_data.to_csv(path_to_input_data + well_name + '_adapt_input.csv')

Построение только тех данных, которые будут использоваться в модели

In [ ]:
if created_input_data_type == 0:
    qliq = {'Объемный дебит жидкости (СУ)':['Объемный дебит жидкости (СУ)']}
    pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)']}
    pressure_wh = {'Рлин ТМ (Ш)': ['Рлин ТМ (Ш)']}
    pressure_bf = {'Рбуф (Ш)': ['Рбуф (Ш)']}
    temp_intake = {'Температура на приеме насоса (пласт. жидкость) (СУ)': ['Температура на приеме насоса (пласт. жидкость) (СУ)']}
    frequencies = {'Частота, Гц':
                   ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)']}
    choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
    power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
    voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
    cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}
    gor = {'ГФ (СУ)':['ГФ (СУ)']}
    wc = {'Процент обводненности (СУ)':['Процент обводненности (СУ)']}
    all_banches = [qliq, gor,wc, pressure_intake, pressure_wh, pressure_bf,
                   temp_intake, frequencies, choke,power,voltage,cos]
    
    pw.create_report_html(created_input_data, all_banches, path_to_input_data + well_name + '_adapt_input_report.html')
    

# Генерация данных для восстановления дебитов

In [ ]:
dir_name_with_input_data = 'adapt_input_2019_11_05_21_21_03' + '\\'
input_data_file_name = '507_adapt_input'
dir_name_with_calculated_data = 'adaptation_2019_11_05_21_36_18' + '\\'
calculated_data_file_name = '507_adapt_1'

Загрузка данных адаптации после расчета

In [ ]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

Чтение входных данных для адаптации и объединение данных

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv')
input_data.index = input_data['Время']
del input_data['Время']
all_data = input_data.join(calculated_data, how = 'outer')

all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков для данных адаптации

In [ ]:
adapt_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', low_memory = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(adapt_data_traces, plot_file_path, True)

In [ ]:
if created_input_data_type == 0:
    qliq = {'Объемный дебит жидкости':['Объемный дебит жидкости (СУ)', 'Q ж, м3/сут (Модель)']}
    calibrs = {'Калибровки по напору и мощности':
           ['К. калибровки по напору - множитель (Модель)',
               'К. калибровки по мощности - множитель (Модель)']}
    metrics = {'Метрики расчета':
           ['Значение функции ошибки (Модель)']}
    gor = {'ГФ (Модель)':['ГФ (Модель)']}
    wc = {'Обводненность, %':['Обв, % (Модель)']}
    pressure_intake = {'Давление на приеме': ['Давление на приеме насоса (пласт. жидкость) (СУ)', 'P прием ЭЦН, атм (Модель)']}
    pressure_wh = {'Рлин': ['Рлин ТМ (Ш)', 'P лин., атм (Модель)']}
    pressure_bf = {'Рбуф': ['Рбуф (Ш)', 'P буф., атм (Модель)']}
    temp_intake = {'Температура на приеме насоса': ['Температура на приеме насоса (пласт. жидкость) (СУ)',
                                                                          'T прием ЭЦН, C (Модель)']}
    frequencies = {'Частота, Гц':
                   ['F вращ ТМ (Ш)', 'Выходная частота ПЧ (СУ)', 'F тока, ГЦ (Модель)']}
    choke = {'Размер штуцера, мм': ['Dшт (Ш)']}
    power = {'Активная мощность (СУ)':['Активная мощность (СУ)']}
    true_power = {'Мощность, передаваемая СУ (Модель)':['Мощность, передаваемая СУ (Модель)']}
    voltage = {'Напряжение на выходе ТМПН (СУ)':['Напряжение на выходе ТМПН (СУ)']}
    cos = {'Коэффициент мощности (СУ)':['Коэффициент мощности (СУ)']}

    efficiency = {'КПД ЭЦН, д.ед.':
              ['КПД ЭЦН, д.ед. (Модель)']}
    dif_pressure = {'Перепад давления в ЭЦН, атм':
                ['Перепад давления в ЭЦН, атм (Модель)']}
    all_banches = [qliq,calibrs,metrics, gor,wc, pressure_intake, pressure_wh, pressure_bf,
                   temp_intake, frequencies, choke,power,true_power, voltage,cos, efficiency, dif_pressure]
    
    pw.create_report_html(all_data, all_banches, path_to_work_dir + dir_name_with_calculated_data + 
                          well_name + '_adapt_report.html')

Работа с калибровками для восстановления дебитов

In [ ]:
calibr_data = calculated_data[['К. калибровки по напору - множитель (Модель)',
                               'К. калибровки по мощности - множитель (Модель)']]
calibr_data = prep.mark_df_columns(calibr_data, 'Подготовленные')
#calibr_data = calibr_data.resample('3h').mean()
#calibr_data = calibr_data.interpolate()

In [ ]:
calibr_data.head()

In [ ]:
try_check = calibr_data.copy()
try_check['Время'] = try_check.index
lenth = len(try_check['Время'])
try_check.index = range(lenth)
try_check = try_check[(try_check.index) % 2 == 0]
try_check = try_check.interpolate()

empty = pd.DataFrame({'empty': list(range(lenth))})
result = empty.join(try_check, how = 'outer')
result.index = calibr_data.index
result = result.interpolate()
result['Время'] = result.index
result.index = result['Время']
del result['empty']
del result['Время']

In [ ]:
calibr_data = result.copy()

In [ ]:
all_data = all_data.join(calibr_data, how = 'inner')

In [ ]:
created_input_data = all_data.copy()

Генерация входных данных для модели для восстановления дебитов

In [ ]:
if created_input_data_type == 1:
    input_data_dir_name = 'restore_input_' + time_mark
    path_to_input_data = path_to_work_dir + input_data_dir_name + '\\'
    if create_input_data:
        try:
            os.mkdir(path_to_work_dir + input_data_dir_name)
        except:
            pass
    created_input_data = edited_data_cs.join(chess_data, how = 'outer')
    created_input_data = created_input_data.join(calibr_data, how ='inner')
    created_input_data = prep.cut_df(created_input_data, left_boundary, right_boundary)
    plot_file_path = path_to_input_data + well_name + '_restore_input.html'
    input_data_traces = pw.create_traces_list_for_all_columms(created_input_data, 'lines+markers', low_memory = True)
    pw.plot_subplots(input_data_traces, plot_file_path, True)
    created_input_data.to_csv(path_to_input_data + well_name + '_restore_input.csv')

# Работа с данными после восстановления дебитов

In [ ]:
dir_name_with_input_data = 'restore_input_2019_11_05_21_21_03' + '\\'
input_data_file_name = '507_restore_input'
dir_name_with_calculated_data = 'restore_2019_11_05_22_23_01' + '\\'
calculated_data_file_name = '507_restore_1'

Загрузка данных после восстановления дебитов

In [ ]:
calculated_data = prep.load_calculated_data_from_csv(path_to_work_dir + dir_name_with_calculated_data +
                                                calculated_data_file_name +  '.csv')

Объединение входных и выходных данным модели

In [ ]:
input_data = pd.read_csv(path_to_work_dir + dir_name_with_input_data + input_data_file_name +  '.csv', parse_dates = True, index_col = 'Время')
all_data = input_data.join(calculated_data, how = 'outer')
all_data.to_csv(path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.csv' )

Построение графиков восстановления дебитов

In [ ]:
input_data_traces = pw.create_traces_list_for_all_columms(all_data, 'lines+markers', use_gl = True)
plot_file_path = path_to_work_dir + dir_name_with_calculated_data + well_name + '_calc_and_input' +  '.html'
pw.plot_subplots(input_data_traces, plot_file_path, True)

# Сведение данных адаптации и восстановления

In [ ]:
path_to_adapt_dir = 'adaptation_2019_11_05_21_36_18' + '\\'
path_to_restore_dir = 'restore_2019_11_05_22_23_01' + '\\'

Загрузка и слияние данных адаптации и восстановления

In [ ]:
adapt_data_with_input = pd.read_csv(path_to_work_dir + path_to_adapt_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
adapt_data_with_input = prep.mark_df_columns(adapt_data_with_input, 'ADAPT')
restore_data_with_input = pd.read_csv(path_to_work_dir + path_to_restore_dir + well_name + '_calc_and_input' + '.csv' , parse_dates = True, index_col = 'Время')
restore_data_with_input = prep.mark_df_columns(restore_data_with_input, 'RESTORE')
overall_data = adapt_data_with_input.join(restore_data_with_input, how = 'outer')


Заключительная обработка данных

In [ ]:
overall_data['Активная мощность (СУ) (ADAPT)'] = overall_data['Активная мощность (СУ) (ADAPT)'] * 1000
overall_data['Загрузка двигателя (СУ) (ADAPT)'] = overall_data['Загрузка двигателя (СУ) (ADAPT)'] / 100
overall_data['Относительная ошибка расчетов (Q ж), %'] = postp.relative_error_perc(overall_data['Объемный дебит жидкости (СУ) (ADAPT)'],
                                                                      overall_data['Q ж, м3/сут (Модель) (RESTORE)'])
overall_data['Относительная ошибка расчетов (N акт), %'] = postp.relative_error_perc(overall_data['Активная мощность (СУ) (ADAPT)'],
                                                                      overall_data['Мощность, передаваемая СУ (Модель) (RESTORE)'])

Подготовка вывода в отчет

In [ ]:
liquid_rates = {'Дебиты': 
                ['Qж ТМ (Ш) (ADAPT)','Объемный дебит жидкости (СУ) (ADAPT)', 'Q ж, м3/сут (Модель) (ADAPT)',
                  'Q ж, м3/сут (Модель) (RESTORE)'] }

calibrs = {'Калибровки по напору и мощности':
           ['К. калибровки по напору - множитель (Модель) (ADAPT)',
               'К. калибровки по мощности - множитель (Модель) (ADAPT)', 
                  'К. калибровки по напору - множитель (Модель) (RESTORE)',
               'К. калибровки по мощности - множитель (Модель) (RESTORE)']}

gor_wc = {'ГФ, м3/м3 и Обводненность, %':
          ['ГФ (Модель) (ADAPT)', 'Обв, % (Модель) (ADAPT)',
                  'ГФ (Модель) (RESTORE)', 'Обв, % (Модель) (RESTORE)']}

temperatures = {'Температура, С':
                ['T устья, С (Модель) (ADAPT)', 'T устья, С (Модель) (RESTORE)', 
                'T прием ЭЦН, C (Модель) (ADAPT)', 'T прием ЭЦН, C (Модель) (RESTORE)']}

pressures_up = {'Давления (Устьевое и буферное), атм':
                ['Рбуф (Ш) (ADAPT)', 'P буф., атм (Модель) (ADAPT)', 'P буф., атм (Модель) (RESTORE)',
             'Рлин ТМ (Ш) (ADAPT)', 'P лин., атм (Модель) (ADAPT)', 'P лин., атм (Модель) (RESTORE)']}

pressures_down = {'Давления (На приеме), атм':
                  ['P прием ЭЦН, атм (Модель) (ADAPT)', 'P прием ЭЦН, атм (Модель) (RESTORE)']}
                 
frequencies = {'Частота, Гц':
               ['F вращ ТМ (Ш) (ADAPT)', 'Выходная частота ПЧ (СУ) (ADAPT)',
               'F тока, ГЦ (Модель) (ADAPT)', 'F тока, ГЦ (Модель) (RESTORE)']}

beam_sizes = {'Размер штуцера, мм':
              ['Dшт (Ш) (ADAPT)', 'Dшт (Ш) (RESTORE)']}

powers = {'Активная мощность, Вт':
          ['Активная мощность (СУ) (ADAPT)', 'Мощность, передаваемая СУ (Модель) (ADAPT)', 'Мощность, передаваемая СУ (Модель) (RESTORE)']}
                       
currents= {'Токи, А':
           ['Ток фазы А (СУ) (ADAPT)', 'I, А (Модель) (ADAPT)', 'I, А (Модель) (RESTORE)']}
               
loads = {'Загрузка, д.ед.':
         ['Загрузка двигателя (Модель) (ADAPT)', 'Загрузка двигателя (Модель) (RESTORE)']}

efficiency = {'КПД ЭЦН, д.ед.':
              ['КПД ЭЦН, д.ед. (Модель) (ADAPT)', 'КПД ЭЦН, д.ед. (Модель) (RESTORE)']}

metrics = {'Метрики расчета':
           ['Значение функции ошибки (Модель) (ADAPT)',
                  'Значение функции ошибки (Модель) (RESTORE)',
          'Относительная ошибка расчетов (Q ж), %',
          'Относительная ошибка расчетов (N акт), %']}

dif_pressure = {'Перепад давления в ЭЦН, атм':
                ['Перепад давления в ЭЦН, атм (Модель) (ADAPT)', 'Перепад давления в ЭЦН, атм (Модель) (RESTORE)']}

all_banches = [liquid_rates,calibrs, metrics, gor_wc, temperatures,pressures_up, 
               pressures_down,frequencies, beam_sizes,powers, currents,loads,efficiency, dif_pressure]

Создание сводного отчета

In [ ]:
plot_file_path = path_to_work_dir + path_to_restore_dir + well_name + '_adapt_and_restore_report' +  '.html'
pw.create_report_html(overall_data, all_banches, plot_file_path)

Построение напорной характеристики ЭЦН

In [ ]:
q_esp_nom_m3day = 320
head_esp_nom_m = 1200

In [ ]:
import sys
import pandas as pd
import os
sys.path.append('C:\\Users\\olegk\\Documents\\')
import unifloc_vba.description_generated.python_api as python_api
path_to_addin = os.getcwd()
path_to_addin = path_to_addin.replace('unifloc\\sandbox\\uTools', 'unifloc_vba\\UniflocVBA_7.xlam')
UniflocVBA = python_api.API(path_to_addin)
h_m = []
efficency_perc = []
power_wt = []
q_m3day = list(range(1, int(q_esp_nom_m3day * 1.8),10))
pump_id = UniflocVBA.calc_ESP_id_by_rate(q_esp_nom_m3day)
num_stages = int(head_esp_nom_m / UniflocVBA.calc_ESP_head_m(q_esp_nom_m3day, pump_id = pump_id))
for i in q_m3day:
    h_m.append(UniflocVBA.calc_ESP_head_m(i,num_stages = num_stages,  pump_id = pump_id) / 100)
    power_wt.append(UniflocVBA.calc_ESP_power_W(i, num_stages = num_stages, pump_id = pump_id) / 1000)
    efficency_perc.append(UniflocVBA.calc_ESP_eff_fr(i, num_stages = num_stages, pump_id = pump_id) * 100)
esp_curve = pd.DataFrame({'Напор, м/100': h_m, 'Мощность, кВт': power_wt, 'КПД, %.': efficency_perc})
esp_curve.index = q_m3day



In [ ]:
esp_curve.head()

In [ ]:
esp_traces = pw.create_traces_list_for_all_columms(esp_curve, chosen_mode='lines+markers', use_gl=False)
pw.plot_func(esp_traces, 'Характеристика ЭЦН', 'Характеристика ЭЦН.html')

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
overall_data_with_calibr_gaps = overall_data[overall_data['К. калибровки по напору - множитель (Модель) (ADAPT)'] !=
                                            overall_data['К. калибровки по напору - множитель (Модель) (RESTORE)']]
coefficient_of_dermination_qliq =  r2_score(overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (ADAPT)'],
                                            overall_data_with_calibr_gaps['Q ж, м3/сут (Модель) (RESTORE)'])
print('coefficient_of_dermination_qliq=' + str(coefficient_of_dermination_qliq))

In [ ]:
print('Средняя относительная ошибка Q ж, %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (Q ж), %'].abs().mean()))
print('Средняя относительная ошибка N акт, %: ' + str(overall_data_with_calibr_gaps['Относительная ошибка расчетов (N акт), %'].abs().mean()))

In [ ]:
overall_data['Относительная ошибка расчетов, %'] = ((overall_data['Q ж, м3/сут (Модель, вход) (ADAPT)'] - \
overall_data['Q ж, м3/сут (Модель, вход) (RESTORE)']) / overall_data['Q ж, м3/сут (Модель, вход) (ADAPT)']) * 100

only_error = overall_data[overall_data['Коэффициент калибровки по напору - множитель (Модель, вход) (ADAPT)'] != overall_data['Коэффициент калибровки по напору - множитель (Модель, вход) (RESTORE)']]
only_error = only_error['Относительная ошибка расчетов, %']

only_error = only_error.dropna()
only_error = only_error.abs()

only_error.mean()
check_r2 = overall_data[['Q ж, м3/сут (Модель, вход) (ADAPT)','Q ж, м3/сут (Модель, вход) (RESTORE)']]

check_r2 = check_r2[check_r2['Q ж, м3/сут (Модель, вход) (ADAPT)'] != check_r2['Q ж, м3/сут (Модель, вход) (RESTORE)']]

check_r2 = check_r2.dropna()
check_r2.head()
from sklearn.metrics import r2_score

coefficient_of_dermination = r2_score(check_r2['Q ж, м3/сут (Модель, вход) (ADAPT)'], check_r2['Q ж, м3/сут (Модель, вход) (RESTORE)'])

coefficient_of_dermination